In [48]:
# use ansible to let argocd setup platform (minio, postgres, mlflow)

In [32]:
export PATH=/work/.local/bin:$PATH
export PYTHONUSERBASE=/work/.local
export ANSIBLE_CONFIG=/work/MLSysEngOps-NYU25Spring/ansible/ansible.cfg
export ANSIBLE_ROLES_PATH=roles

In [29]:
cd /work/MLSysEngOps-NYU25Spring/ansible
ansible-playbook -i inventory.yml argocd/argocd_add_platform.yml


PLAY [Mount persistent volume and deploy MLflow platform with MinIO] ***********

TASK [Gathering Facts] *********************************************************
ok: [node1]

TASK [Detect filesystem on /dev/vdb] *******************************************
ok: [node1]

TASK [Format /dev/vdb as ext4 on first use] ************************************
skipping: [node1]

TASK [Make root mount directory] ***********************************************
ok: [node1]

TASK [Mount volume & ensure /etc/fstab] ****************************************
ok: [node1]

TASK [Make MinIO data dir] *****************************************************
ok: [node1]

TASK [Make Postgres data dir] **************************************************
ok: [node1]

PLAY [Deploy MLflow platform via ArgoCD & Helm with MinIO secret handling] *****

TASK [Gathering Facts] *********************************************************
ok: [node1]

TASK [Get ArgoCD admin password from Kubernetes secret] *********************

In [35]:
cd /work/MLSysEngOps-NYU25Spring/workflows
cat build-initial.yaml

apiVersion: argoproj.io/v1alpha1
kind: Workflow
metadata:
  generateName: kaniko-pvc-build-
spec:
  entrypoint: kaniko-build-flow
  volumeClaimTemplates:
  - metadata:
      name: workdir
    spec:
      accessModes: [ "ReadWriteOnce" ]
      resources:
        requests:
          storage: 1Gi

  templates:
  - name: kaniko-build-flow
    steps:
      - - name: clone-repo
          template: git-clone
      - - name: kaniko-build
          template: kaniko

  - name: git-clone
    container:
      image: alpine/git
      command: [sh, -c]
      args:
        - |
          MAX_RETRIES=5
          i=1

          while [ $i -le $MAX_RETRIES ]; do
            echo "Attempt $i of $MAX_RETRIES: Cloning repo..."
            git clone --branch app https://github.com/zuchuandatou/MLSysEngOps-NYU25Spring.git /mnt/workspace && break
            echo "Git clone failed. Retrying in 5 seconds..."
            i=$((i + 1))
            sleep 5
          done

          if [ ! -d /mnt/workspace/.git ]; 

In [40]:
cd /work/MLSysEngOps-NYU25Spring/ansible
ansible-playbook -i inventory.yml argocd/workflow_build_init.yml


PLAY [Run Argo Workflow from GitHub Repo] **************************************

TASK [Gathering Facts] *********************************************************
ok: [node1]

TASK [Clone or update gourmetgram-iac repo] ************************************
changed: [node1]

TASK [Submit Argo Workflow] ****************************************************
changed: [node1]

TASK [Extract Workflow Name] ***************************************************
ok: [node1]

TASK [Wait for workflow to complete (success or fail)] *************************
changed: [node1]

TASK [Get final workflow result] ***********************************************
changed: [node1]

TASK [Display workflow phase] **************************************************
ok: [node1] => 
  msg: 'Workflow kaniko-pvc-build-9xghg finished with status: Succeeded'

TASK [Fail if workflow did not succeed] ****************************************
skipping: [node1]

PLAY RECAP ***************************************************

In [41]:
cd /work/MLSysEngOps-NYU25Spring/ansible
ansible-playbook -i inventory.yml argocd/argocd_add_staging.yml


PLAY [Deploy Project37 Staging via ArgoCD & Helm] ******************************

TASK [Gathering Facts] *********************************************************
ok: [node1]

TASK [Get ArgoCD admin password] ***********************************************
changed: [node1]

TASK [Decode ArgoCD password] **************************************************
changed: [node1]

TASK [Login to ArgoCD] *********************************************************
ok: [node1]

TASK [Detect external IP starting with 10.56] **********************************
ok: [node1]

TASK [Create project37-staging namespace if missing] ***************************
ok: [node1]

TASK [Check if ArgoCD app exists] **********************************************
ok: [node1]

TASK [Create ArgoCD Helm app if not exists] ************************************
skipping: [node1]

TASK [Update ArgoCD Helm app if exists] ****************************************
changed: [node1]

TASK [Display ArgoCD app status] *****************

In [ ]:
cd /work/MLSysEngOps-NYU25Spring/ansible
ansible-playbook -i inventory.yml argocd/argocd_add_canary.yml

In [ ]:
cd /work/MLSysEngOps-NYU25Spring/ansible
ansible-playbook -i inventory.yml argocd/argocd_add_prod.yml

In [ ]:
cd /work/MLSysEngOps-NYU25Spring/ansible
ansible-playbook -i inventory.yml argocd/workflow_templates_apply.yml